<a href="https://colab.research.google.com/github/rajeevranjan665/project/blob/main/maincode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [ ]:
credits.head()

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.head()

In [ ]:
print(movies.columns)

In [ ]:
movies = movies['original_language'].value_counts()

In [ ]:
movies.info()

In [ ]:
import pandas as pd

# Load the datasets
movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/tmdb_5000_credits.csv')

# Merge the datasets on 'title'
movies = movies.merge(credits, on='title')

# Now extract the required columns
movies = movies[['movie_id', 'cast', 'crew', 'title', 'overview', 'genres', 'keywords']]

# Check the output
print(movies.head())


In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast

In [ ]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
        return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['cast'][0]

In [ ]:
from typing import Counter
def convert3(obj):
    L=[]
    Counter = 0
    for i in ast.literal_eval(obj):
      if Counter != 3:
        L.append(i['name'])
        Counter+=1
      else:
        break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies.head()

In [ ]:
movies['crew'][0]

In [ ]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
      if i['job'] == 'Director':
        L.append(i['name'])
      else:
        break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x: x.split() if isinstance(x, str) else x)

In [ ]:
movies.head()

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x] if isinstance(x, list) else x)
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x] if isinstance(x, list) else x)
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x] if isinstance(x, list) else x)
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ","") for i in x] if isinstance(x, list) else x)

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
new_df = movies[['movie_id','title','tags']]
new_df

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x) if isinstance(x, list) else "")
new_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vector[0]

In [ ]:
cv.get_feature_names_out()

In [ ]:
import nltk

In [ ]:
!pip install nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
      y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity[0]

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)


In [ ]:
recommend('Batman Begins')